# Module 04: KV-Aware Routing

> **Goal**: Understand how Dynamo's KV-aware router directs requests to workers with cached prefixes for optimal performance.

---

## Overview

LLM inference benefits enormously from **KV cache reuse**. When users send follow-up messages in a conversation, or when multiple users ask similar questions, the prompt prefixes are often identical. Reusing cached KV blocks avoids redundant computation.

**KV-aware routing** solves a key problem: with multiple workers, how do we route requests to the worker that already has the relevant prefix cached?

### How It Works

1. **Workers publish KV events**: When blocks are stored/evicted, workers notify the router
2. **Router tracks cache state**: Maintains a radix tree of which worker has which prefix blocks
3. **Smart routing**: Routes requests to workers with matching cached blocks

### Event Plane for KV Events

The router receives KV cache events via an event plane. NATS is the default.

| Event Plane | Flag | Use Case |
|-------------|------|----------|
| **NATS** (default) | `--event-plane nats` | Distributed, persistent events |
| **ZMQ** | `--event-plane zmq` | Lower latency, local only |

### Monitoring KV Events

Open a **separate terminal** and run:

```bash
docker run --rm -it --network host natsio/nats-box nats sub ">" --server localhost:4222
```

This subscribes to ALL NATS messages. Keep it running while you work through this notebook to see KV events (BlockStored, BlockEvicted) in real-time.

---

## Step 1: Verify GPUs and Infrastructure

We need 2 GPUs for two workers, plus etcd and NATS running.

In [1]:
import urllib.request
import json
import socket
import subprocess
import time

print("=" * 60)
print("GPU CHECK")
print("=" * 60)

result = subprocess.run(
    ['nvidia-smi', '--query-gpu=index,name,memory.free', '--format=csv,noheader'],
    capture_output=True, text=True, timeout=5
)

gpus = result.stdout.strip().split('\n')
print(f"\nFound {len(gpus)} GPU(s):")
for gpu in gpus:
    print(f"  {gpu}")

if len(gpus) >= 2:
    print("\n✓ Two GPUs available!")
else:
    print("\n⚠️  Need 2 GPUs for this demo")

print("\n" + "=" * 60)
print("INFRASTRUCTURE CHECK")
print("=" * 60)

def check_port(port):
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(2)
        result = sock.connect_ex(('localhost', port))
        sock.close()
        return result == 0
    except:
        return False

try:
    with urllib.request.urlopen("http://localhost:2379/health", timeout=5) as resp:
        print("✓ etcd: Running")
except:
    print("✗ etcd: Not running (docker start dynamo-etcd)")

print(f"{'✓' if check_port(4222) else '✗'} NATS: {'Running' if check_port(4222) else 'Not running'}")
print(f"{'✓' if check_port(8222) else '✗'} NATS Monitor: {'Running' if check_port(8222) else 'Not running'}")

GPU CHECK

Found 2 GPU(s):
  0, NVIDIA GeForce RTX 5090, 2302 MiB
  1, NVIDIA GeForce RTX 5090, 2302 MiB

✓ Two GPUs available!

INFRASTRUCTURE CHECK
✓ etcd: Running
✓ NATS: Running
✓ NATS Monitor: Running


---

## Step 2: Stop Any Existing Dynamo Processes

In [2]:
%%bash
echo "=== Finding Dynamo processes ==="
pgrep -af 'python -m dynamo' || echo "No Dynamo processes found"

echo -e "\n=== Finding processes on port 8000 ==="
fuser -v 8000/tcp 2>&1 || echo "No processes on port 8000"

echo -e "\n=== Sending SIGTERM to Dynamo processes ==="
pkill -f 'python -m dynamo' && echo "Sent SIGTERM to Dynamo processes" || echo "No Dynamo processes to kill"
sleep 2

echo -e "\n=== Sending SIGKILL to any remaining Dynamo processes ==="
pkill -9 -f 'python -m dynamo' && echo "Sent SIGKILL to remaining processes" || echo "No remaining processes"

echo -e "\n=== Force killing processes on port 8000 ==="
fuser -k -9 8000/tcp 2>&1 && echo "Killed processes on port 8000" || echo "No processes on port 8000 to kill"

sleep 3

echo -e "\n=== Verifying cleanup ==="
pgrep -af 'python -m dynamo' && echo "WARNING: Some processes still running!" || echo "✓ All Dynamo processes stopped"
fuser -v 8000/tcp 2>&1 && echo "WARNING: Port 8000 still in use!" || echo "✓ Port 8000 is free"
echo -e "\n✓ Cleanup complete"

=== Finding Dynamo processes ===


3290245 python -m dynamo.frontend --router-mode kv
3290246 python -m dynamo.sglang --model-path Qwen/Qwen3-0.6B --host 0.0.0.0 --attention-backend flashinfer --page-size 16
3290250 python -m dynamo.sglang --model-path Qwen/Qwen3-0.6B --host 0.0.0.0 --attention-backend flashinfer --page-size 16

=== Finding processes on port 8000 ===
                     USER        PID ACCESS COMMAND
8000/tcp:            root      3290245 F.... python

=== Sending SIGTERM to Dynamo processes ===
Sent SIGTERM to Dynamo processes

=== Sending SIGKILL to any remaining Dynamo processes ===
Sent SIGKILL to remaining processes

=== Force killing processes on port 8000 ===
No processes on port 8000 to kill

=== Verifying cleanup ===
✓ All Dynamo processes stopped
✓ Port 8000 is free

✓ Cleanup complete


---

## Step 3: Launch Two Workers with KV-Aware Routing

For **KV-aware routing** to work, we need **multiple workers**. The router chooses which worker based on which one has the relevant prefix cached.

**Key flags:**
- Frontend: `--router-mode kv` - Enable KV-aware routing
- Workers: `--page-size 16` - Set the KV cache block size (the router discovers this from workers' MDC)
- **Workers: `--kv-events-config` - CRITICAL! Enables KV cache event publishing**

**How KV Events Flow:**
```
Worker 1 (GPU 0)                    Worker 2 (GPU 1)
     │                                   │
     │ ZMQ publish                       │ ZMQ publish
     │ tcp://*:5557                      │ tcp://*:5558
     ▼                                   ▼
┌─────────────────────────────────────────────────┐
│             NATS JetStream                      │
│         (kv-events topic)                       │
└───────────────────────┬─────────────────────────┘
                        │ subscribe
                        ▼
               ┌─────────────────┐
               │    KV Router    │
               │ (radix tree of  │
               │  cached blocks) │
               └─────────────────┘
```

Without `--kv-events-config`, workers don't advertise their cached blocks → router sees 0 cached blocks!

```
                    Request: "Hello, how are you?"
                              │
                              ▼
                    ┌───────────────────┐
                    │     KV Router     │
                    │                   │
                    │ Which worker has  │
                    │ this prefix?      │
                    └─────────┬─────────┘
                              │
              ┌───────────────┴───────────────┐
              ▼                               ▼
    ┌─────────────────┐             ┌─────────────────┐
    │   Worker 1      │             │   Worker 2      │
    │   (GPU 0)       │             │   (GPU 1)       │
    │ Has: "Hello"    │             │ Has: "What is"  │
    └────────┬────────┘             └─────────────────┘
             │
             ▼
    Route to Worker 1 (cache hit!)
```

In [3]:
%%bash
MODEL="Qwen/Qwen3-0.6B"

echo "============================================================"
echo "LAUNCHING TWO WORKERS WITH KV-AWARE ROUTING"
echo "============================================================"

echo ""
echo "[1/3] Starting Frontend (--router-mode kv)..."
python -m dynamo.frontend --router-mode kv > /tmp/dynamo_frontend.log 2>&1 &
echo "      PID: $!"

echo ""
echo "[2/3] Starting Worker 1 (GPU 0) with KV events on port 5557..."
CUDA_VISIBLE_DEVICES=0 python -m dynamo.sglang \
    --model-path $MODEL \
    --host 0.0.0.0 \
    --attention-backend flashinfer \
    --page-size 16 \
    --kv-events-config '{"publisher":"zmq","topic":"kv-events","endpoint":"tcp://*:5557"}' \
    > /tmp/dynamo_worker1.log 2>&1 &
echo "      PID: $!"

echo ""
echo "[3/3] Starting Worker 2 (GPU 1) with KV events on port 5558..."
CUDA_VISIBLE_DEVICES=1 python -m dynamo.sglang \
    --model-path $MODEL \
    --host 0.0.0.0 \
    --attention-backend flashinfer \
    --page-size 16 \
    --kv-events-config '{"publisher":"zmq","topic":"kv-events","endpoint":"tcp://*:5558"}' \
    > /tmp/dynamo_worker2.log 2>&1 &
echo "      PID: $!"

echo ""
echo "✓ All processes started"
echo "  Logs: /tmp/dynamo_frontend.log, /tmp/dynamo_worker1.log, /tmp/dynamo_worker2.log"
echo ""
echo "⏳ Wait ~60-90s for models to load..."

LAUNCHING TWO WORKERS WITH KV-AWARE ROUTING

[1/3] Starting Frontend (--router-mode kv)...
      PID: 3294908

[2/3] Starting Worker 1 (GPU 0) with KV events on port 5557...
      PID: 3294909

[3/3] Starting Worker 2 (GPU 1) with KV events on port 5558...
      PID: 3294910

✓ All processes started
  Logs: /tmp/dynamo_frontend.log, /tmp/dynamo_worker1.log, /tmp/dynamo_worker2.log

⏳ Wait ~60-90s for models to load...


---

## Step 4: Wait for Workers to Register

In [4]:
print("Waiting for workers to start...")
print("(This may take ~60-90 seconds)\n")

MAX_WAIT = 180
INTERVAL = 10
elapsed = 0

while elapsed < MAX_WAIT:
    try:
        with urllib.request.urlopen('http://localhost:8000/health', timeout=5) as response:
            health = json.loads(response.read())
            instances = health.get('instances', [])
            
            print(f"[{elapsed}s] Frontend: ✓ | Workers: {len(instances)}")
            
            if len(instances) >= 2:
                print(f"\n✓ Two workers ready!")
                for inst in instances:
                    print(f"  • {inst.get('component', '?')}/{inst.get('endpoint', '?')}")
                break
    except:
        print(f"[{elapsed}s] Waiting...")
    
    time.sleep(INTERVAL)
    elapsed += INTERVAL

if elapsed >= MAX_WAIT:
    print(f"\n⚠️  Timeout. Check logs: tail /tmp/dynamo_worker1.log")

Waiting for workers to start...
(This may take ~60-90 seconds)

[0s] Waiting...


[10s] Frontend: ✓ | Workers: 1
[20s] Frontend: ✓ | Workers: 1
[30s] Frontend: ✓ | Workers: 1
[40s] Frontend: ✓ | Workers: 1
[50s] Frontend: ✓ | Workers: 1
[60s] Frontend: ✓ | Workers: 1
[70s] Frontend: ✓ | Workers: 1
[80s] Frontend: ✓ | Workers: 1
[90s] Frontend: ✓ | Workers: 3

✓ Two workers ready!
  • backend/generate
  • backend/generate
  • kv-router/generate


---

## Step 5: Check NATS Server Status

Let's see what's happening on NATS before we send requests.

In [5]:
print("=" * 60)
print("NATS SERVER STATUS")
print("=" * 60)

try:
    with urllib.request.urlopen("http://localhost:8222/varz", timeout=5) as resp:
        varz = json.loads(resp.read())
        print(f"\nServer: {varz.get('server_name', '?')}")
        print(f"Uptime: {varz.get('uptime', '?')}")
        print(f"Connections: {varz.get('connections', 0)}")
        print(f"Messages In: {varz.get('in_msgs', 0):,}")
        print(f"Messages Out: {varz.get('out_msgs', 0):,}")
except Exception as e:
    print(f"Error: {e}")

print("\n" + "=" * 60)
print("JETSTREAM STREAMS")
print("=" * 60)

try:
    with urllib.request.urlopen("http://localhost:8222/jsz?streams=true", timeout=5) as resp:
        jsz = json.loads(resp.read())
        print(f"\nStreams: {jsz.get('streams', 0)}")
        print(f"Consumers: {jsz.get('consumers', 0)}")
        
        for account in jsz.get('account_details', []):
            for stream in account.get('stream_detail', []):
                name = stream.get('name', '?')
                msgs = stream.get('state', {}).get('messages', 0)
                print(f"  • {name}: {msgs} messages")
except Exception as e:
    print(f"Error: {e}")

NATS SERVER STATUS

Server: NBOM62WZSE5VHSNVZDZHEPUP4355H2LINXONTVB4DDOFY7AMRSMNJYQW
Uptime: 10h2m9s
Connections: 8
Messages In: 3,670
Messages Out: 17,147

JETSTREAM STREAMS

Streams: 1
Consumers: 1
  • namespace-dynamo-component-backend-kv-events: 4 messages


---

## Step 6: Demonstrate KV-Aware Routing

Now let's send requests and observe routing behavior.

**Watch your nats-box terminal** for `BlockStored` events!

---

## Troubleshooting: KV Events Not Appearing

If you don't see BlockStored events in NATS, run this diagnostic script.

In [6]:
def send_request(prompt):
    """Send inference request to Dynamo."""
    url = "http://localhost:8000/v1/chat/completions"
    payload = {
        "model": "Qwen/Qwen3-0.6B",
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 30
    }
    start = time.time()
    try:
        req = urllib.request.Request(
            url,
            data=json.dumps(payload).encode(),
            headers={'Content-Type': 'application/json'}
        )
        with urllib.request.urlopen(req, timeout=120) as resp:
            result = json.loads(resp.read())
            elapsed = time.time() - start
            return result["choices"][0]["message"]["content"], elapsed
    except Exception as e:
        return f"Error: {e}", 0

PROMPT_A = "Explain the theory of relativity in simple terms."
PROMPT_B = "What are the main differences between Python and JavaScript programming languages?"

print("=" * 60)
print("KV-AWARE ROUTING DEMO")
print("=" * 60)
print("\n🔍 Watch your nats-box terminal for BlockStored events!\n")

# Request 1: New prompt, creates KV cache
print(f"[1] '{PROMPT_A[:40]}...'")
resp, t = send_request(PROMPT_A)
print(f"    Time: {t:.2f}s | Response: {resp[:50]}...\n")
print("    ⏳ Waiting 3s for KV events to propagate...")
time.sleep(3)

# Request 2: Same prompt, should hit cache on same worker
print(f"[2] SAME PROMPT (should route to same worker)")
resp, t = send_request(PROMPT_A)
print(f"    Time: {t:.2f}s | Response: {resp[:50]}...\n")
print("    ⏳ Waiting 3s for KV events to propagate...")
time.sleep(3)

# Request 3: Different prompt
print(f"[3] '{PROMPT_B}'")
resp, t = send_request(PROMPT_B)
print(f"    Time: {t:.2f}s | Response: {resp[:50]}...\n")
print("    ⏳ Waiting 3s for KV events to propagate...")
time.sleep(3)

# Request 4: Repeat prompt B
print(f"[4] SAME AS [3] (should route to same worker)")
resp, t = send_request(PROMPT_B)
print(f"    Time: {t:.2f}s | Response: {resp[:50]}...\n")

print("=" * 60)
print("Check your nats-box terminal for the KV events!")

KV-AWARE ROUTING DEMO

🔍 Watch your nats-box terminal for BlockStored events!

[1] 'Explain the theory of relativity in simp...'


    Time: 3.54s | Response: <think>
Okay, so I need to explain the theory of r...

    ⏳ Waiting 3s for KV events to propagate...
[2] SAME PROMPT (should route to same worker)
    Time: 0.42s | Response: <think>
Okay, so I need to explain the theory of r...

    ⏳ Waiting 3s for KV events to propagate...
[3] 'What are the main differences between Python and JavaScript programming languages?'
    Time: 3.72s | Response: <think>
Okay, let me try to figure out the main di...

    ⏳ Waiting 3s for KV events to propagate...
[4] SAME AS [3] (should route to same worker)
    Time: 0.36s | Response: <think>
Okay, so the user is asking about the main...

Check your nats-box terminal for the KV events!


---

## Step 7: Verify KV-Aware Routing Worked

Let's check the frontend logs and NATS state to confirm cache hits.

In [7]:
import re

print("=" * 60)
print("FRONTEND LOGS: KV ROUTER DECISIONS")
print("=" * 60)
print("\nLast 20 routing decisions from /tmp/dynamo_frontend.log:\n")

try:
    with open('/tmp/dynamo_frontend.log', 'r') as f:
        content = f.read()
    
    # Find all routing decision lines
    lines = content.split('\n')
    routing_lines = [l for l in lines if 'cached blocks' in l or 'Selected worker' in l]
    
    # Show last 20 relevant lines
    for line in routing_lines[-20:]:
        # Clean up ANSI codes
        clean = re.sub(r'\x1b\[[0-9;]*m', '', line)
        # Extract key info
        if 'cached blocks' in clean:
            # Extract: worker_id, cached blocks, logit
            match = re.search(r'worker_id=(\d+).*with (\d+) cached blocks.*: ([\d.]+)', clean)
            if match:
                worker_id = match.group(1)[-4:]  # Last 4 digits
                cached = match.group(2)
                logit = match.group(3)
                cache_indicator = "✓ HIT" if int(cached) > 0 else "  miss"
                print(f"  Worker ...{worker_id}: {cached} cached blocks, logit={logit} {cache_indicator}")
        elif 'Selected worker' in clean:
            match = re.search(r'worker_id=(\d+).*cached blocks: (\d+)', clean)
            if match:
                worker_id = match.group(1)[-4:]
                cached = match.group(2)
                print(f"  → SELECTED: ...{worker_id} (cached: {cached})\n")

except FileNotFoundError:
    print("  Frontend log not found")
except Exception as e:
    print(f"  Error: {e}")

print("\n" + "=" * 60)
print("NATS JETSTREAM STATUS")
print("=" * 60)

try:
    with urllib.request.urlopen("http://localhost:8222/jsz?streams=true", timeout=5) as resp:
        jsz = json.loads(resp.read())
        
        for account in jsz.get('account_details', []):
            for stream in account.get('stream_detail', []):
                name = stream.get('name', '?')
                if 'kv-events' in name:
                    msgs = stream.get('state', {}).get('messages', 0)
                    consumers = stream.get('state', {}).get('consumer_count', 0)
                    print(f"\n  Stream: {name}")
                    print(f"  BlockStored events: {msgs}")
                    print(f"  Consumers (routers): {consumers}")
except Exception as e:
    print(f"  Error: {e}")

print("\n" + "=" * 60)
print("INTERPRETATION")
print("=" * 60)
print("""
✓ HIT = Router found cached blocks for this prefix on this worker
  miss = No cached blocks (new prompt or cache evicted)

If you see cache hits on repeat prompts, KV-aware routing is working!
The router correctly routes requests to workers with cached prefixes.
""")

FRONTEND LOGS: KV ROUTER DECISIONS

Last 20 routing decisions from /tmp/dynamo_frontend.log:

  Worker ...1412: 0 cached blocks, logit=2.188   miss
  Worker ...1415: 0 cached blocks, logit=2.188   miss
  Worker ...1415: 1 cached blocks, logit=1.188 ✓ HIT
  Worker ...1412: 0 cached blocks, logit=2.188   miss
  Worker ...1415: 0 cached blocks, logit=2.250   miss
  Worker ...1412: 0 cached blocks, logit=2.250   miss
  Worker ...1412: 1 cached blocks, logit=1.250 ✓ HIT
  Worker ...1415: 0 cached blocks, logit=2.250   miss

NATS JETSTREAM STATUS

  Stream: namespace-dynamo-component-backend-kv-events
  BlockStored events: 6
  Consumers (routers): 1

INTERPRETATION

✓ HIT = Router found cached blocks for this prefix on this worker
  miss = No cached blocks (new prompt or cache evicted)

If you see cache hits on repeat prompts, KV-aware routing is working!
The router correctly routes requests to workers with cached prefixes.



---

## Cleanup

In [8]:
%%bash
echo "Stopping Dynamo processes..."
pkill -f 'python -m dynamo' 2>/dev/null || true
sleep 10
echo "✓ Cleanup complete"

echo ""
nvidia-smi --query-gpu=index,memory.used,memory.free --format=csv

Stopping Dynamo processes...
✓ Cleanup complete

index, memory.used [MiB], memory.free [MiB]
0, 4 MiB, 32106 MiB
1, 4 MiB, 32106 MiB


---

## Summary: NATS in Dynamo

### Architecture

```
┌────────────────────────────────────────────────────────────────────┐
│  ┌─────────────┐     ┌─────────────┐     ┌─────────────────────┐   │
│  │   Frontend  │────▶│  KV Router  │────▶│      Workers        │   │
│  │   (:8000)   │ TCP │             │ TCP │  (GPU 0, GPU 1)     │   │
│  └─────────────┘     └──────┬──────┘     └──────────┬──────────┘   │
│                             │                       │              │
│                             │ subscribe             │ publish      │
│                             ▼                       ▼              │
│                    ┌────────────────────────────────────────┐      │
│                    │           NATS JetStream               │      │
│                    │  • KV cache events (BlockStored)       │      │
│                    │  • Router uses to pick best worker     │      │
│                    └────────────────────────────────────────┘      │
│                                                                    │
│  Request data → TCP (fast)                                         │
│  KV events → NATS (coordination)                                   │
└────────────────────────────────────────────────────────────────────┘
```

### Monitoring Commands

```bash
# Subscribe to all NATS messages
docker run --rm -it --network host natsio/nats-box nats sub ">" --server localhost:4222

# List JetStream streams
docker run --rm --network host natsio/nats-box nats stream ls --server localhost:4222

# View all messages in a JetStream stream
docker run -it --rm --network host natsio/nats-box nats stream view namespace-dynamo-component-backend-kv-events --all --server localhost:4222
```

### Key Takeaways

1. **NATS is for events, not data**: KV cache *events* flow through NATS; actual data uses TCP/NIXL
2. **KV-aware routing**: Router subscribes to events to know which worker has which prefix
3. **Same prefix → same worker**: Requests with similar prompts route to workers with cached prefixes
4. **nats-box is your friend**: Simple CLI for real-time NATS monitoring